<a href="https://colab.research.google.com/github/lovnishverma/Python-Getting-Started/blob/main/ONNX_Model_with_Your_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [81]:
!pip install skl2onnx onnx

In [82]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
import onnx

In [83]:
# Step 1: Load the dataset
iris = pd.read_csv("https://raw.githubusercontent.com/lovnishverma/datasets/refs/heads/main/iriswithheaders.csv")
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [84]:
# Step 2: Features and target
X = iris[["sepal_length", "sepal_width", "petal_length", "petal_width"]]
y = iris["species"]

In [85]:
# Step 3: Encode target labels (scikit-learn automatically handles string Labels(Target) in classification problems, but not for categorical features.)
le = LabelEncoder()
y_encoded = le.fit_transform(y)  # classes_: ['setosa', 'versicolor', 'virginica']

In [86]:
# Optional: Save class mapping
import json
with open("class_labels.json", "w") as f:
    json.dump(le.classes_.tolist(), f)

In [87]:
# Step 4: Train the model
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = LogisticRegression(max_iter=200)
model.fit(X_train, y_train)

LogisticRegression(max_iter=200)

In [88]:
# Step 6: Test The model on test input
# prediction = model.predict([[5.7, 3.2, 5.2, 1.9]])
print(le.inverse_transform([prediction[0]]))  # ['virginica']
# print(prediction[0])

['virginica']


/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [89]:
# Step 5: Convert to ONNX
initial_type = [('float_input', FloatTensorType([None, 4]))]
onnx_model = convert_sklearn(model, initial_types=initial_type)

In [90]:
# Step 6: Save ONNX model
with open("iris_model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

print("✅ Model saved as iris_model.onnx")


✅ Model saved as iris_model.onnx


Step-by-Step: Predict with ONNX Model

In [91]:
!pip install onnxruntime


In [92]:
import onnxruntime as rt
import numpy as np
import json

# Load ONNX model
sess = rt.InferenceSession("iris_model.onnx")

# Get input/output names
input_name = sess.get_inputs()[0].name
output_name = sess.get_outputs()[0].name

# Load class label mapping
with open("class_labels.json", "r") as f:
    class_labels = json.load(f)

# Prepare test input
features = np.array([[5.7, 3.2, 5.2, 1.9]], dtype=np.float32)

# Run inference
prediction = sess.run([output_name], {input_name: features})

# Get predicted class index
predicted_index = int(prediction[0][0])

# Map index to class name
predicted_class = class_labels[predicted_index]

print(f"✅ Predicted class index: {predicted_index}")
print(f"🌸 Predicted class label: {predicted_class}")


✅ Predicted class index: 2
🌸 Predicted class label: virginica
